In [1]:
from subprocess import check_output
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from sklearn.model_selection import  train_test_split
import time #helper libraries
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from numpy import newaxis
import numpy as np
import pandas as pd
import math
import talib

Using TensorFlow backend.


In [7]:
fundamentals = pd.read_csv('./fundamentals.csv')
stock_fundamentals = fundamentals.loc[fundamentals['Ticker Symbol'] == 'AAPL'].copy()
stock_fundamentals.head()

,Unnamed: 0,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,...,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding
8,8,AAPL,2013-09-28,3.622300e+10,-1.949000e+09,1.156000e+09,30.0,-8.165000e+09,0.0,93.0,...,7.328600e+10,4.365800e+10,1.235490e+11,8.345100e+10,2.070000e+11,1.709100e+11,0.0,2013.0,40.03,9.252311e+08
9,9,AAPL,2014-09-27,4.864900e+10,-6.452000e+09,9.800000e+08,35.0,-9.571000e+09,0.0,40.0,...,6.853100e+10,6.344800e+10,1.115470e+11,1.202920e+11,2.318390e+11,1.827950e+11,0.0,2014.0,6.49,6.087827e+09
10,10,AAPL,2015-09-26,6.067100e+10,-3.124000e+09,1.285000e+09,45.0,-1.124700e+10,0.0,52.0,...,8.937800e+10,8.061000e+10,1.193550e+11,1.709900e+11,2.903450e+11,2.337150e+11,0.0,2015.0,9.28,5.753664e+09
11,11,AAPL,2016-09-24,5.932100e+10,1.044000e+09,1.348000e+09,36.0,-1.273400e+10,0.0,85.0,...,1.068690e+11,7.900600e+10,1.282490e+11,1.934370e+11,3.216860e+11,2.156390e+11,0.0,2016.0,8.35,5.471497e+09


In [18]:
prices = pd.read_csv('./prices-split-adjusted.csv') #get data
stock = prices.loc[prices['symbol'] == 'AAPL'].copy() #get only GOOG prices
stock["y"] = (stock.close - stock.close.shift(1))/stock.close.shift(1) #create returns (called y since that's what we're trying to predict)
LOOK_BACK = 5
#Add features
for i in range(1, LOOK_BACK+1):
    #Add column for returns of n days ago (e.g. yesterday's returns, the returns of the day before yesterday...)
    stock[str(i) + " Days Ago"] = stock["y"].shift(i)
technical_indicators = []
stock = stock[['y'] + [str(i) + " Days Ago" for i in range(1, LOOK_BACK + 1)] + technical_indicators].copy().dropna() #drop the other columns

#Scale and separate test and train data
TEST_SIZE = 0.33
train_stop = math.floor((1-TEST_SIZE)*len(stock.index)) #index of where test data starts and training data stops
train = stock.loc[stock.index[0:train_stop]]
for col in stock.columns: 
    scaler = MinMaxScaler(feature_range=(0,1))
    scaler.fit(train[[col]]) #!!!!!!!!!!!!!!!! ONLY FIT SCALER ON TRAINING DATA, FITTING ON TEST DATA AS WELL IS CHEATING !!!!!!!!!!!!
    stock[col] = scaler.transform(stock[[col]]) #apply scaler to all data 

#Actually separate test data
stock = np.array(stock.values)
x_train = stock[:train_stop, 1:]
x_test = stock[train_stop:, 1:]
y_train = stock[:train_stop, 0]
y_test = stock[train_stop:, 0]
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

#Define model architecture (copied from someone else's model, will optimize with a genetic algorithm later)
model = Sequential()
model.add(LSTM(input_shape=(1, x_train.shape[2]),units = 50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100,return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.add(Activation('linear'))

#Compile, fit on training data, and evaluate on test data
model.compile(loss='mse', optimizer='rmsprop')
model.fit(x_train, y_train, batch_size = 128, epochs=10, validation_split = 0.05, verbose = 0)
model.evaluate(x_test, y_test, verbose = 0)
y_predict = model.predict(x_test)
returns = []
for i in range(len(x_test) - 1):
    if y_predict[i][0] > x_test[i][0][-1]:
        returns.append((y_test[i]-x_test[i][0][-1])/x_test[i][0][-1])
    else:
        returns.append(0)
sharpe = np.mean(returns)/np.std(returns)
print("Sharpe Ratio: ", sharpe)

Sharpe Ratio:  0.345680053080649
